## Rover Project Test Notebook
This notebook contains the functions from the lesson and provides the scaffolding you need to test out your mapping methods.  The steps you need to complete in this notebook for the project are the following:

* First just run each of the cells in the notebook, examine the code and the results of each.
* Run the simulator in "Training Mode" and record some data. Note: the simulator may crash if you try to record a large (longer than a few minutes) dataset, but you don't need a ton of data, just some example images to work with.   
* Change the data directory path (2 cells below) to be the directory where you saved data
* Test out the functions provided on your data
* Write new functions (or modify existing ones) to report and map out detections of obstacles and rock samples (yellow rocks)
* Populate the `process_image()` function with the appropriate steps/functions to go from a raw image to a worldmap.
* Run the cell that calls `process_image()` using `moviepy` functions to create video output
* Once you have mapping working, move on to modifying `perception.py` and `decision.py` to allow your rover to navigate and map in autonomous mode!

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

**Run the next cell to get code highlighting in the markdown cells.**

In [ ]:
%%HTML
<style> code {background-color : orange !important;} </style>

In [ ]:
%matplotlib inline
#%matplotlib qt # Choose %matplotlib qt to plot to an interactive window (note it may show up behind your browser)
# Make some of the relevant imports
import cv2 # OpenCV for perspective transform
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import scipy.misc as misc # For saving images as needed
import glob  # For reading in a list of images from a folder
import imageio
from IPython.display import Image
from IPython.display import display
imageio.plugins.ffmpeg.download()

## Quick Look at the Data
There's some example data provided in the `test_dataset` folder.  This basic dataset is enough to get you up and running but if you want to hone your methods more carefully you should record some data of your own to sample various scenarios in the simulator.  

Next, read in and display a random image from the `test_dataset` folder

In [ ]:
path = '../test_dataset/IMG/*'
img_list = glob.glob(path)
# Grab a random image and display it
idx = np.random.randint(0, len(img_list)-1)
image = mpimg.imread(img_list[idx])
plt.imshow(image)

## Calibration Data
Read in and display example grid and rock sample calibration images.  You'll use the grid for perspective transform and the rock image for creating a new color selection that identifies these samples of interest. 

In [ ]:
# In the simulator you can toggle on a grid on the ground for calibration
# You can also toggle on the rock samples with the 0 (zero) key.  
# Here's an example of the grid and one of the rocks
example_grid = '../calibration_images/example_grid1.jpg'
example_rock = '../calibration_images/example_rock1.jpg'
grid_img = mpimg.imread(example_grid)
rock_img = mpimg.imread(example_rock)

fig = plt.figure(figsize=(12,3))
plt.subplot(121)
plt.imshow(grid_img)
plt.subplot(122)
plt.imshow(rock_img)

## Perspective Transform

Define the perspective transform function from the lesson and test it on an image.

In [ ]:
# Define a function to perform a perspective transform
# I've used the example grid image above to choose source points for the
# grid cell in front of the rover (each grid cell is 1 square meter in the sim)
# Define a function to perform a perspective transform
def perspect_transform(img, src, dst):
           
    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(img, M, (img.shape[1], img.shape[0]))# keep same size as input image
    
    return warped

# Define calibration box in source (actual) and destination (desired) coordinates
# These source and destination points are defined to warp the image
# to a grid where each 10x10 pixel square represents 1 square meter
# The destination box will be 2*dst_size on each side
dst_size = 5 
# Set a bottom offset to account for the fact that the bottom of the image 
# is not the position of the rover but a bit in front of it
# this is just a rough guess, feel free to change it!
bottom_offset = 6
source = np.float32([[14, 140], [301 ,140],[200, 96], [118, 96]])
destination = np.float32([[image.shape[1]/2 - dst_size, image.shape[0] - bottom_offset],
                  [image.shape[1]/2 + dst_size, image.shape[0] - bottom_offset],
                  [image.shape[1]/2 + dst_size, image.shape[0] - 2*dst_size - bottom_offset], 
                  [image.shape[1]/2 - dst_size, image.shape[0] - 2*dst_size - bottom_offset],
                  ])
warped = perspect_transform(grid_img, source, destination)
plt.imshow(warped)
#scipy.misc.imsave('../output/warped_example.jpg', warped)

## Color Thresholding
Define the color thresholding function from the lesson and apply it to the warped image

**TODO:** Ultimately, you want your map to not just include navigable terrain but also obstacles and the positions of the rock samples you're searching for.  Modify this function or write a new function that returns the pixel locations of obstacles (areas below the threshold) and rock samples (yellow rocks in calibration images), such that you can map these areas into world coordinates as well.  
**Hints and Suggestion:** 
* For obstacles you can just invert your color selection that you used to detect ground pixels, i.e., if you've decided that everything above the threshold is navigable terrain, then everthing below the threshold must be an obstacle!


* For rocks, think about imposing a lower and upper boundary in your color selection to be more specific about choosing colors.  You can investigate the colors of the rocks (the RGB pixel values) in an interactive matplotlib window to get a feel for the appropriate threshold range (keep in mind you may want different ranges for each of R, G and B!).  Feel free to get creative and even bring in functions from other libraries.  Here's an example of [color selection](http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_colorspaces/py_colorspaces.html) using OpenCV.  

* **Beware However:** if you start manipulating images with OpenCV, keep in mind that it defaults to `BGR` instead of `RGB` color space when reading/writing images, so things can get confusing.

## Find out color threshold for rocks

In [ ]:
# These two images are cut from the camera pictures
rock1_fn, rock2_fn = '../calibration_images/rock1_pixels.png', '../calibration_images/rock2_pixels.png'
rock1 = misc.imread(rock1_fn)
rock2 = misc.imread(rock2_fn)
r1 = Image(filename=rock1_fn)
r2 = Image(filename=rock2_fn)
display(r1, r2)

In [ ]:
# Find out the min/max for r, g, b values
rock_r = np.concatenate((rock1[:,:,0].flatten(), rock2[:,:,0].flatten()))
rock_g = np.concatenate((rock1[:,:,1].flatten(), rock2[:,:,1].flatten()))
rock_b = np.concatenate((rock1[:,:,2].flatten(), rock2[:,:,2].flatten()))

rock_thresh_lo = (min(rock_r), min(rock_g), min(rock_b))
rock_thresh_hi = (max(rock_r), max(rock_g), max(rock_b)) 

rock_thresh_lo, rock_thresh_hi

In [ ]:
# Identify pixels above and below given thresholds
def color_thresh(img, lo=(0, 0, 0), hi=(255, 255, 255)):
    color_select = np.zeros_like(img[:,:,0])
    mask = (img[:,:,0] >= lo[0]) & (img[:,:,1] >= lo[1]) & (img[:,:,2] >= lo[2]) & \
           (img[:,:,0] <= hi[0]) & (img[:,:,1] <= hi[1]) & (img[:,:,2] <= hi[2])
    color_select[mask] = 1
    return color_select

navigable_thresh_lo = (161, 161, 161)

thresh_navigable = color_thresh(grid_img, lo=navigable_thresh_lo)
thresh_obstacle = color_thresh(grid_img, hi=navigable_thresh_lo)
thresh_rock = color_thresh(rock_img, lo=rock_thresh_lo, hi=rock_thresh_hi)

map_navigable = perspect_transform(thresh_navigable, source, destination)
map_obstacle = perspect_transform(thresh_obstacle, source, destination)
map_rock = perspect_transform(thresh_rock, source, destination)

# Plot navigable terrain, obstacles, and rocks:
# - row 1 - original images
# - row 2 - thresholded images
# - row 3 - perpective transform to map

fig = plt.figure(figsize=(12,6))
plt.subplot(331)
plt.imshow(grid_img)
plt.subplot(332)
plt.imshow(grid_img)
plt.subplot(333)
plt.imshow(rock_img)
plt.subplot(334)
plt.imshow(thresh_navigable, cmap='gray')
plt.subplot(335)
plt.imshow(thresh_obstacle, cmap='gray')
plt.subplot(336)
plt.imshow(thresh_rock, cmap='gray')
plt.subplot(337)
plt.imshow(map_navigable, cmap='gray')
plt.subplot(338)
plt.imshow(map_obstacle, cmap='gray')
plt.subplot(339)
plt.imshow(map_rock, cmap='gray')

#scipy.misc.imsave('../output/warped_threshed.jpg', threshed*255)

## Coordinate Transformations
Define the functions used to do coordinate transforms and apply them to an image.

In [ ]:
# Define a function to convert from image coords to rover coords
def rover_coords(binary_img):
    # Identify nonzero pixels
    ypos, xpos = binary_img.nonzero()
    # Calculate pixel positions with reference to the rover position being at the 
    # center bottom of the image.  
    x_pixel = -(ypos - binary_img.shape[0]).astype(np.float)
    y_pixel = -(xpos - binary_img.shape[1]/2 ).astype(np.float)
    return x_pixel, y_pixel

# Define a function to convert to radial coords in rover space
def to_polar_coords(x_pixel, y_pixel):
    # Convert (x_pixel, y_pixel) to (distance, angle) 
    # in polar coordinates in rover space
    # Calculate distance to each pixel
    dist = np.sqrt(x_pixel**2 + y_pixel**2)
    # Calculate angle away from vertical for each pixel
    angles = np.arctan2(y_pixel, x_pixel)
    return dist, angles

# Define a function to map rover space pixels to world space
def rotate_pix(xpix, ypix, yaw):
    # Convert yaw to radians
    yaw_rad = yaw * np.pi / 180
    xpix_rotated = (xpix * np.cos(yaw_rad)) - (ypix * np.sin(yaw_rad))
                            
    ypix_rotated = (xpix * np.sin(yaw_rad)) + (ypix * np.cos(yaw_rad))
    # Return the result  
    return xpix_rotated, ypix_rotated

def translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale): 
    # Apply a scaling and a translation
    xpix_translated = (xpix_rot / scale) + xpos
    ypix_translated = (ypix_rot / scale) + ypos
    # Return the result  
    return xpix_translated, ypix_translated

# Define a function to apply rotation and translation (and clipping)
# Once you define the two functions above this function should work
def pix_to_world(xpix, ypix, xpos, ypos, yaw, world_size, scale):
    # Apply rotation
    xpix_rot, ypix_rot = rotate_pix(xpix, ypix, yaw)
    # Apply translation
    xpix_tran, ypix_tran = translate_pix(xpix_rot, ypix_rot, xpos, ypos, scale)
    # Perform rotation, translation and clipping all at once
    x_pix_world = np.clip(np.int_(xpix_tran), 0, world_size - 1)
    y_pix_world = np.clip(np.int_(ypix_tran), 0, world_size - 1)
    # Return the result
    return x_pix_world, y_pix_world

# Grab another random image
idx = np.random.randint(0, len(img_list)-1)
image = mpimg.imread(img_list[idx])
warped = perspect_transform(image, source, destination)
threshed = color_thresh(warped, lo=navigable_thresh_lo)

# Calculate pixel values in rover-centric coords and distance/angle to all pixels
xpix, ypix = rover_coords(threshed)
dist, angles = to_polar_coords(xpix, ypix)
mean_dir = np.mean(angles)

# Do some plotting
fig = plt.figure(figsize=(12,9))
plt.subplot(221)
plt.imshow(image)
plt.subplot(222)
plt.imshow(warped)
plt.subplot(223)
plt.imshow(threshed, cmap='gray')
plt.subplot(224)
plt.plot(xpix, ypix, '.')
plt.ylim(-160, 160)
plt.xlim(0, 160)
arrow_length = 100
x_arrow = arrow_length * np.cos(mean_dir)
y_arrow = arrow_length * np.sin(mean_dir)
plt.arrow(0, 0, x_arrow, y_arrow, color='red', zorder=2, head_width=10, width=2)

## Read in saved data and ground truth map of the world
The next cell is all setup to read your saved data into a `pandas` dataframe.  Here you'll also read in a "ground truth" map of the world, where white pixels (pixel value = 1) represent navigable terrain.  

After that, we'll define a class to store telemetry data and pathnames to images.  When you instantiate this class (`data = Databucket()`) you'll have a global variable called `data` that you can refer to for telemetry and map data within the `process_image()` function in the following cell.  


In [ ]:
# Import pandas and read in csv file as a dataframe
import pandas as pd
# Change the path below to your data directory
# If you are in a locale (e.g., Europe) that uses ',' as the decimal separator
# change the '.' to ','
df = pd.read_csv('../test_dataset/robot_log.csv', delimiter=';', decimal='.')
csv_img_list = df["Path"].tolist() # Create list of image pathnames
# Read in ground truth map and create a 3-channel image with it
ground_truth = mpimg.imread('../calibration_images/map_bw.png')
ground_truth_3d = np.dstack((ground_truth*0, ground_truth*255, ground_truth*0)).astype(np.float)

# Creating a class to be the data container
# Will read in saved data from csv file and populate this object
# Worldmap is instantiated as 200 x 200 grids corresponding 
# to a 200m x 200m space (same size as the ground truth map: 200 x 200 pixels)
# This encompasses the full range of output position values in x and y from the sim
class Databucket():
    def __init__(self):
        self.images = csv_img_list  
        self.xpos = df["X_Position"].values
        self.ypos = df["Y_Position"].values
        self.yaw = df["Yaw"].values
        self.count = 0 # This will be a running index
        self.worldmap = np.zeros((200, 200, 3)).astype(np.float)
        self.ground_truth = ground_truth_3d # Ground truth worldmap

# Instantiate a Databucket().. this will be a global variable/object
# that you can refer to in the process_image() function below
data = Databucket()

## Write a function to process stored images

Modify the `process_image()` function below by adding in the perception step processes (functions defined above) to perform image analysis and mapping.  The following cell is all set up to use this `process_image()` function in conjunction with the `moviepy` video processing package to create a video from the images you saved taking data in the simulator.  

In short, you will be passing individual images into `process_image()` and building up an image called `output_image` that will be stored as one frame of video.  You can make a mosaic of the various steps of your analysis process and add text as you like (example provided below).  



To start with, you can simply run the next three cells to see what happens, but then go ahead and modify them such that the output video demonstrates your mapping process.  Feel free to get creative!

In [ ]:
# Helper functions to create a custom mosaic arrangement from several images

# Find out where to put images, first columns then row, so they don't overlap
def image_positions(num_columns, *imgs):
    next_y = 0 
    for i in range(len(imgs)):
        (x, y) = (x + imgs[i - 1].shape[1], y) if i % num_columns else (0, next_y)
        next_y = max(next_y, y + imgs[i].shape[0])
        yield (y, x)

# Put images in the layout defined by the number of columns
def arrange_images(num_columns, *imgs):
    pos = list(image_positions(num_columns, *imgs))
    size = lambda dim: max(pos[i][dim] + imgs[i].shape[dim] for i in range(len(imgs)))
    output_image = np.zeros((size(0), size(1), 3))
    for i in range(len(imgs)):
        output_image[pos[i][0]:pos[i][0] + imgs[i].shape[0],
                     pos[i][1]:pos[i][1] + imgs[i].shape[1]] = imgs[i]

    return output_image

In [ ]:
# Define a function to pass stored images to
# reading rover position and yaw angle from csv file
# This function will be used by moviepy to create an output video
def process_image(img):

    dst_size = 5 
    world_size, scale = data.worldmap.shape[0], 2 * dst_size

    # Define source and destination for perspective transform
    bottom_offset = 6
    source = np.float32([[14, 140], [301 ,140],[200, 96], [118, 96]])
    destination = np.float32([[img.shape[1]/2 - dst_size, img.shape[0] - bottom_offset],
                              [img.shape[1]/2 + dst_size, img.shape[0] - bottom_offset],
                              [img.shape[1]/2 + dst_size, img.shape[0] - 2*dst_size - bottom_offset], 
                              [img.shape[1]/2 - dst_size, img.shape[0] - 2*dst_size - bottom_offset]])
    
    xpos, ypos, yaw = data.xpos[data.count], data.ypos[data.count], data.yaw[data.count]
    
    # Navigable terrain 
    nav_thresh = color_thresh(img, lo=navigable_thresh_lo)
    nav_warped = perspect_transform(nav_thresh, source, destination)
    nav_x_rover, nav_y_rover = rover_coords(nav_warped)
    nav_x_world, nav_y_world = pix_to_world(nav_x_rover, nav_y_rover, xpos, ypos, yaw, world_size, scale)
       
    # Obstacles 
    obs_thresh = color_thresh(img, hi=navigable_thresh_lo)
    obs_warped = perspect_transform(obs_thresh, source, destination)
    obs_x_rover, obs_y_rover = rover_coords(obs_warped)
    obs_x_world, obs_y_world = pix_to_world(obs_x_rover, obs_y_rover, xpos, ypos, yaw, world_size, scale)

    # Rock samples
    rock_thresh = color_thresh(img, lo=rock_thresh_lo, hi=rock_thresh_hi)
    rock_warped = perspect_transform(rock_thresh, source, destination)
    rock_x_rover, rock_y_rover = rover_coords(rock_warped)
    rock_x_world, rock_y_world = pix_to_world(rock_x_rover, rock_y_rover, xpos, ypos, yaw, world_size, scale)

    # Update world map and overlay ground truth map
    data.worldmap[obs_y_world, obs_x_world, 0] = 255
    data.worldmap[rock_y_world, rock_x_world, 1] = 255
    data.worldmap[nav_y_world, nav_x_world, 2] = 255
    # Resolve conflicts: prefer navigable, when it is both navigable and obstacle
    data.worldmap[data.worldmap[:, :, 2].nonzero(), 0] = 0
    
    map_overlay = np.flipud(cv2.addWeighted(data.worldmap, 1, data.ground_truth, 1, 0))
    
    # Thresholded image from rover POV
    thresh = np.concatenate([obs_thresh[:,:,np.newaxis]*128,
                             rock_thresh[:,:,np.newaxis]*255,
                             nav_thresh[:,:,np.newaxis]*128], axis=2)
    
    # Thresholded image from above
    warped = np.concatenate([obs_warped[:,:,np.newaxis]*128,
                             rock_warped[:,:,np.newaxis]*255,
                             nav_warped[:,:,np.newaxis]*128], axis=2)
    
    # Combine images
    output_image = arrange_images(2, img, thresh, warped, map_overlay)

    if data.count < len(data.images) - 1:
        data.count += 1 # Keep track of the index in the Databucket()
    
    return output_image

## Test how it looks

In [ ]:
from IPython.display import Image

data = Databucket()
data.count = np.random.randint(0, len(img_list)-1)
image = mpimg.imread(img_list[data.count])

test_image = process_image(image)
fn = '../output/test_image.jpg'
misc.imsave(fn, test_image)
Image(filename=fn)

## Make a video from processed image data
Use the [moviepy](https://zulko.github.io/moviepy/) library to process images and create a video.
  

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from moviepy.editor import ImageSequenceClip


# Define pathname to save the output video
output = '../output/test_mapping.mp4'
data = Databucket() # Re-initialize data in case you're running this cell multiple times
clip = ImageSequenceClip(data.images, fps=60) # Note: output video will be sped up because 
                                          # recording rate in simulator is fps=25
new_clip = clip.fl_image(process_image) #NOTE: this function expects color images!!
%time new_clip.write_videofile(output, audio=False)

### This next cell should function as an inline video player
If this fails to render the video, try running the following cell (alternative video rendering method).  You can also simply have a look at the saved mp4 in your `/output` folder

In [ ]:
from IPython.display import HTML
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))

### Below is an alternative way to create a video in case the above cell did not work.

In [ ]:
import io
import base64
video = io.open(output, 'r+b').read()
encoded_video = base64.b64encode(video)
HTML(data='''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded_video.decode('ascii')))